In [ ]:
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

dfAuthor = pd.DataFrame()
dfAuthor = pd.read_csv('DBPediaAuthors.csv', sep=',', low_memory=False)


display(dfAuthor.head())

In [ ]:
#Search DBPedia for Authors
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
      SELECT DISTINCT *
      WHERE
        {
          ?s a dbo:Writer .
        }
        LIMIT 10000
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

display(results)


In [ ]:
import json
import pandas as pd

import urllib3
import urllib
import csv

import re

def createMetaData( data, uriPath , dfMeta):
    try:
        data = json.loads(data.decode('utf-8'))
        detailedData = data[''.join(['http://dbpedia.org/resource/', uriPath])]
        
        skipShortKeys = {'22-rdf-syntax-ns#type', 'rdf-schema#comment', 'thumbnail', 'wikiPageRevisionID', 
                #, 'wikiPageExternalLink', 'prov#wasDerivedFrom', 'wikiPageID', 'signature', 'bgcolor', 'quote', 
                    'imageSize', 'align', 'bgcolour', 'colwidth', 'salign', '\'\'\'name\'\'\'_'}
        
        #display(detailedData)
        
        for key in detailedData.keys():
            shortKey = key.split('/')[-1]
            if shortKey not in skipShortKeys:
                values = []
                values.append(key)
                for content in detailedData[key]:
                    if 'datatype' in content:
                        #display(content['datatype'])
                        values.append(content['datatype'].split('/')[-1])
                        values.append(content['datatype'])
                    else:
                        values.append("XMLSchema#string")
                        values.append("http://www.w3.org/2001/XMLSchema#string")
                dfMeta[shortKey] = pd.Series(values)
        
        return dfMeta
    except:
        return dfMeta

def convertResultToSeries( data, uriPath ):
    try:   
        data = json.loads(data.decode('utf-8'))
        detailedData = data[''.join(['http://dbpedia.org/resource/', uriPath])]
        #display(data)
        author = {}
        skipShortKeys = {'22-rdf-syntax-ns#type', 'rdf-schema#comment', 'thumbnail', 'wikiPageRevisionID', 
                #, 'wikiPageExternalLink', 'prov#wasDerivedFrom', 'wikiPageID', 'signature', 'bgcolor', 'quote', 
                    'imageSize', 'align', 'bgcolour', 'colwidth', 'salign', '\'\'\'name\'\'\'_'}
        author['URI'] = ''.join(['http://dbpedia.org/resource/', uriPath])
    
        for key in detailedData.keys():
            shortKey = key.split('/')[-1]
            if shortKey not in skipShortKeys:
                values = []
                for content in detailedData[key]:
                    if 'lang' in content:
                        if 'en' in content['lang']:
                            values.append(str(content['value']).replace('\\n', '').replace('http://dbpedia.org/resource/', ''))
                    else:
                        value = str(content['value']).replace('\\n', '').replace('http://dbpedia.org/resource/', '')
                        if len(re.findall('[\0-\200]', value)) > 0 :
                            values.append(value)
                author[shortKey.lower()] = '\t'.join(set(values))
        return pd.DataFrame(author, index=[0])
    
    except:
        return pd.DataFrame()
    

http = urllib3.PoolManager()
listSeries = []
counter = 0

dfAuthorMeta = pd.DataFrame()

uri = []
uri.append("URI")
uri.append("URI")
uri.append("http://www.w3.org/2002/07/owl#Thing")
dfAuthorMeta['URI'] = pd.Series(uri)

for result in results["results"]["bindings"]:
    #if result["s"]["value"] not in dfAuthor['identifier'].values:
        uriPath = result["s"]["value"].replace('http://dbpedia.org/resource/', '')
        uriPathQuoted = urllib.parse.quote(uriPath)
        url = "http://dbpedia.org/data/%s.json" % uriPathQuoted
        r = http.request('GET', url)
        dfAuthor = dfAuthor.append(convertResultToSeries(r.data, uriPath), ignore_index=True, sort=True)
        dfAuthorMeta = createMetaData(r.data, uriPath, dfAuthorMeta)
        counter = counter + 1
        if counter > 10:
            counter = 0
            dfAuthorNew = dfAuthorMeta.append(dfAuthor, ignore_index=True, sort=True)
            dfAuthorNew.to_csv('DBPediaAuthorsNew.csv', sep=',', encoding='utf-8', index=True, quotechar='"', quoting=csv.QUOTE_ALL)

dfAuthor = dfAuthorMeta.append(dfAuthor, ignore_index=True, sort=True)
#display(dfAuthor)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
display(dfAuthor)

In [ ]:
works = []
works.append("http://dbpedia.org/ontology/notableWork")
works.append("XMLSchema#string")
works.append("http://www.w3.org/2001/XMLSchema#string")
for index, row in dfAuthor.iterrows():
    try:
        if(index > 2):
            sparql = SPARQLWrapper("http://dbpedia.org/sparql")
            query = "SELECT DISTINCT * WHERE { ?s dbo:author dbr:%s.} LIMIT 10000" % row['URI'].replace('http://dbpedia.org/resource/', '')
            sparql.setQuery(query)
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()
            work_author = ''
            for result in results["results"]["bindings"]:
                work_author = '\t'.join([work_author, result["s"]["value"].replace('http://dbpedia.org/resource/', '')])
            works.append(work_author)
    except:
        display(query)

dfAuthor['work'] = pd.Series(works)
display(dfAuthor)

In [ ]:
pd.set_option('display.width', 10000)

In [ ]:
#Keep only columns which are at least more than 15% filled
#dfAuthorMeta = dfAuthor.describe()
#dfAuthorMetaCount = dfAuthorMeta.loc[['count']]
#dfAuthorMetaCount = dfAuthorMetaCount.transpose()
#dfSelected = dfAuthorMetaCount[((dfAuthorMetaCount['count']/len(dfAuthorMetaCount)) * 100) > 15]
#dfAuthor = dfAuthor[dfSelected.index]

In [ ]:
#dfAuthor['notableWorks_join'] = dfAuthor['notableWork'].map(str).replace('nan', '') + dfAuthor['notableWorks'].map(str).replace('nan', '')
#dfAuthor['notableWorks_join'] = dfAuthor['notableWorks_join'].map(str) + dfAuthor['notableWork(s)_'].map(str).replace('nan', '')

#display(dfAuthor['notableWorks_join'])

dfAuthor.to_csv('DBPediaAuthorsNew.csv', sep=',', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)


In [ ]:
import csv
#del dfAuthor['1a']
#del dfAuthor['1p']
#del dfAuthor['1pp']
#del dfAuthor['1y']
#del dfAuthor['2a']
#del dfAuthor['2p']
#del dfAuthor['2pp']
#del dfAuthor['2ps']
#del dfAuthor['2y']
#del dfAuthor['3a']
#del dfAuthor['3p']
#del dfAuthor['3pp']
#del dfAuthor['3y']
#del dfAuthor['birthdate'] # birth date (lowercase) is not used (empty)
#del dfAuthor['deathdate'] # death date (lowercase) is not used (empty)
#display(dfAuthor)
#dfAuthor.to_csv('DBPediaAuthors.csv', sep=',', encoding='utf-8', quotechar='"', quoting=csv.QUOTE_ALL)

In [ ]:
from dateutil.parser import parse
import numpy as np

dfAuthorNew = pd.DataFrame()

for index, row in dfAuthor.iterrows():
    if index > 2:
        if(isinstance(row['birthDate'], str)):
            dates = row['birthDate'].split('\t')
            row['birthDate'] = dates[0]
            if '-0-0' in row['birthDate']:
                for date in dates:
                    if not '-0-0' in date:
                        row['birthDate'] = date
            if '-0-0' in row['birthDate']:
                row['birthDate'] = row['birthDate'].replace('-0-0', '')
        if(isinstance(row['birthDate'], float) and np.isnan(row['birthDate'])):
            if(isinstance(row['birth'], float) and ~np.isnan(row['birth'])):
                row['birthDate'] = row['birth']
            elif(isinstance(row['birthYear'], float) and ~np.isnan(row['birthYear'])):
                row['birthDate'] = row['birthYear']
            elif(isinstance(row['dateOfBirth'], float) and ~np.isnan(row['dateOfBirth'])):
                row['birthDate'] = row['dateOfBirth']
        
        if(isinstance(row['deathDate'], str)):
            dates = row['deathDate'].split('\t')
            row['deathDate'] = dates[0]
            if '-0-0' in row['deathDate']:
                for date in dates:
                    if not '-0-0' in date:
                        row['deathDate'] = date
            if '-0-0' in row['deathDate']:
                row['deathDate'] = row['deathDate'].replace('-0-0', '')
                
        if(isinstance(row['deathDate'], float) and np.isnan(row['deathDate'])):
            if(isinstance(row['death'], float) and ~np.isnan(row['death'])):
                row['deathDate'] = row['death']
            elif(isinstance(row['deathYear'], float) and ~np.isnan(row['deathYear'])):
                row['deathDate'] = row['deathYear']
            
    dfAuthorNew = dfAuthorNew.append(row, ignore_index=True, sort=True)

del dfAuthorNew['birth']
del dfAuthorNew['birthYear']
del dfAuthorNew['dateOfBirth']
del dfAuthorNew['death']
del dfAuthorNew['deathYear']
display(dfAuthorNew)

In [ ]:
import csv
dfAuthor.to_csv('DBPediaAuthors.csv', sep=',', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)
display(dfAuthorNew)
dfAuthorNew.to_csv('DBPediaAuthorsNew.csv', sep=',', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)

In [ ]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
display(dfAuthor.head())

In [ ]:
#display(dfAuthor.describe(include="all"))
dfAuthorNew = pd.DataFrame()
listWorks = ["notableWork", "notableWork(s)_", "notableWorks", "notablework", "notableworks", "work"]

for index, row in dfAuthor.iterrows():
    if index > 2:
        row['consolidatedWorks'] = np.nan
        for header in listWorks:
            if(isinstance(row[header], str)):
                works = row[header].split('\t')
                for work in works:
                    if(isinstance(row['consolidatedWorks'], str)):
                        if work not in row['consolidatedWorks']:
                            row['consolidatedWorks'] = row['consolidatedWorks'] + '\t' + work
                    else:
                        row['consolidatedWorks'] = work
        
        dfAuthorNew = dfAuthorNew.append(row, ignore_index=True, sort=True)

In [ ]:
del dfAuthorNew['notableWork']
del dfAuthorNew['notableWork(s)_']
del dfAuthorNew['notableWorks']
del dfAuthorNew['notablework']
del dfAuthorNew['notableworks']
del dfAuthorNew['work']

In [ ]:
import numpy as np

dfAuthorInfo = dfAuthorNew.describe(include="all").transpose()
display(dfAuthorInfo.sort_values(by=['count'], ascending=False))

In [ ]:
import csv
dfAuthorNew.to_csv('DBPediaAuthorsNew.csv', sep=',', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL)